<div dir="rtl">
  <h1>تفريغ - الكتب المُيسّرة</h1>

  <p>مرحبًا بك في تفريغ، أحد منتجات مشروع الكتب المُيسّرة. لاستخدام تفريغ:</p>

  <ol>
    <li>قم بإدخال روابط المواد المطلوب تحويلها من منصة YouTube في حقل "الروابط" وتأكد من فصلها بفاصلة كهذه ","</li>
    <li>قم باختيار لغة المادة في حقل "اللغة"</li>
    <li>قم باختيار النموذج المُراد استخدامه في حقل "النموذج"</li>
    <li>قم بتشغيل الخلية في الأسفل من خلال الضغط على الدائرة المحتوية على السهم</li>
  </ol>

  <p>هذه الخطوات ستقوم بتحويل المواد إلى نصوص باستخدام نماذج Whisper. إذا كنت تريد استخدام <a href="wit.ai">wit.ai</a>، يمكنك وضع المفتاح الخاص بك في حقل wit.</p>

  <p>يمكنك تجربة التفريغ باستخدام نماذج Whisper و تقنيات wit.ai واستخدام التفريغ الأفضل في حالتك. كملاحظ عامة، نماذج Whisper تقوم بتفريغ الهمزات والتشكيلات وعلامات الترقيم بشكل أفضل من wit.ai، لكن wit.ai يُنتج أخطاء إملائية أقل.</p>

  <p>عندما ينتهي التحويل سيتم تنزيل الملفات النصية بشكل تلقائي بصيغة <code>txt</code> و <code>srt</code> وسيكون اسم الملف هو مُعرّف المادة على منصة YouTube كما هو مُوضّح: https://youtu.be/<strong>4h5P7jXvW98</strong>.</p>

  <hr>

  <p>يمكنك متابعة مشروع <strong>الكتب المُيسّرة</strong> والتواصل معنا من خلال:</p>

  <ul>
    <li><a href="https://t.me/ieasybooks">قناتنا على تيليجرام</a></li>
    <li><a href="https://www.youtube.com/@ieasybooks">قناتنا على يوتيوب</a></li>
    <li><a href="https://twitter.com/iieasybooks">حسابنا على تويتر</a></li>
    <li><a href="https://www.facebook.com/ieasybooks">صفحتنا على فيسبوك</a></li>
    <li><a href="https://github.com/ieasybooks">حسابنا على GitHub (للمبرمجين)</a></li>
    <li>بريدنا الالكتروني: easybooksdev@gmail.com</li>
  </ul>
</div>

In [ ]:
print('جارٍ تجهيز بيئة العمل.')

# Setup Tafrigh.
!pip uninstall -y tafrigh
!pip install --no-cache-dir git+https://github.com/ieasybooks/tafrigh -qqq

# Get inputs.
الروابط = 'https://youtu.be/4h5P7jXvW98 https://youtu.be/jpfndVSROpw'  # @param { type: "string" }
اللغة = 'ar'  # @param ["ar", "af", "am", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "en", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "he", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]
النموذج = 'large-v2 (\u0623\u0641\u0636\u0644 \u062F\u0642\u0629)'  # @param ["large-v2 (أفضل دقة)", "medium", "base", "small", "tiny (أقل دقة)"]
wit = ''  # @param { type: "string" }

urls = الروابط
language = اللغة
model = النموذج

if model == 'large-v2 (\u0623\u0641\u0636\u0644 \u062F\u0642\u0629)':
  model = 'large-v2'
elif model == 'tiny (\u0623\u0642\u0644 \u062F\u0642\u0629)':
  model = 'tiny'

# Imports.
import glob
import os

from ctranslate2.converters import TransformersConverter
from google.colab import files
from tafrigh import farrigh, Config

# Setup directories.
output_dir = 'output'
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

if not wit:
  # Convert Transformers model to CTranslate2 format.
  print('جارٍ تنزيل وتحويل النموذج.')

  transforms_converter = TransformersConverter(
    model_name_or_path=f"openai/whisper-{model}",
    copy_files=['tokenizer.json'],
    load_as_float16=True,
  )

  transforms_converter.convert(
    output_dir=f"whisper-{model}-ct2",
    quantization='float16',
    force=True,
  )

# Start Tafrigh.
if wit:
  print('جارٍ تحويل المواد إلى نصوص باستخدام تقنيات wit.ai.')
else:
  print('جارٍ تحويل المواد إلى نصوص باستخدام نماذج Whisper.')

config = Config(
  urls=list(map(str.strip, urls.split(','))),
  model_name_or_ct2_model_path=f"whisper-{model}-ct2",
  wit_client_access_token=wit,
  task='transcribe',
  language=language,
  beam_size=5,
  ct2_compute_type='float16',
  min_words_per_segment=0,
  output_formats=['txt', 'srt'],
  save_yt_dlp_responses=False,
  output_dir=output_dir,
  verbose=False,
)

farrigh(config)

# Download all txt and srt files.
print('جارٍ تنزيل الملفات النصية.')

txt_files = glob.glob(f"{output_dir}/*.txt")
srt_files = glob.glob(f"{output_dir}/*.srt")

for file in txt_files + srt_files:
  files.download(file)